# <center> Predictive modelling with timeseries</center>
# <center> Part 4 - Time series forecasting with Facebook's Prophet</center>

![Image](images/timeseries.jpg)

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# jupyter lab configs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)

from utils import print_errors
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot

# Exercise: Cryptocurrency value prediction

# ETL

### Load the datasets

In [ ]:
bc = pd.read_csv('datasets/crypto.csv', decimal=".").reset_index(drop=True)
bc.Date = pd.to_datetime(bc.Date)
bc.sort_values('Date', inplace=True)
bc.head()
"min max date"
bc.Date.min(), bc.Date.max() 
"min max price"
bc.Price.min(), bc.Price.max() 

In [ ]:
bc.info()

### `NOTE` the Prophet requires that datetime is named `ds` and the target is named `y`

In [ ]:
bc.rename(columns={'Date': 'ds', 'Price': 'y'}, inplace=True)

### Train-test

In [ ]:
# Let's train until 2019 and test on 2020
# split 
X_train = bc[bc.ds<='2019-12-31'][['ds', 'y']]
X_test = bc[bc.ds>'2019-12-31'][['ds']]

### Let's fit using the default configuration

In [ ]:
# fit
model = Prophet()
model.fit(X_train)

In [ ]:
# check the predictions for the training data
pred_train = model.predict(X_train)

# use the trained model to make a forecast
pred_test = model.predict(X_test)

In [ ]:
# check the model's output
pred_test.head(4)

## Plot predictions - test dataset

In [ ]:
# plot forecast
p = model.plot(pred_test)

## Plot predictions - training dataset

In [ ]:
plot_plotly(model, pred_train)

In [ ]:
plot_components_plotly(model, pred_train)

# 🤩 Perfecting the model

# 1. Add events and holidays  

One of the advantages of the Prophet is the ability to add covariates to specific dates. They can be given by you or taken *out-of-the-box* by using the package `holidays`.
For a user-specified input, event dates must be profived for past and future, and we should make a dataframe with the following format: 

In [ ]:
user_events = pd.DataFrame({
  'holiday': ['something_amazing', 'crash', 'something_amazing', 'something_amazing'],
  'ds': pd.to_datetime(['2017-12-17', '2018-12-16', '2019-06-26', '2020-11-15']),
  'lower_window': 0,
  'upper_window': 0})

In [ ]:
user_events

In [ ]:
m3 = Prophet(holidays=user_events)
m3.fit(X_train)
pred_train = m3.predict(X_train)
pred_test = m3.predict(X_test)
fig = m3.plot(pred_test)

### Add calendar holidays

In [ ]:
m4 = Prophet()
m4.add_country_holidays(country_name='DE')
m4.fit(X_train)

In [ ]:
# check holidays included
m4.train_holiday_names

# 2. Configuring the TREND 

## 1. Changepoints

Prophet will use 80% of the data (by default) to identify points where the trend changed abruptly.
We can change this value, but care must be taken as increasing it may lead to overfitting and extreme changes in the nearest predictions.


### 80% changepoints

In [ ]:
fig = model.plot(pred_train)
a = add_changepoints_to_plot(fig.gca(), model, pred_train)

### 90% changepoints

In [ ]:
m9 = Prophet(changepoint_range=0.9)
m9.fit(X_train)
pred_train = m9.predict(X_train)
pred_test = m9.predict(X_test)
fig = m9.plot(pred_train)
a = add_changepoints_to_plot(fig.gca(), m9, pred_train)

### Manually inserting changepoints

In [ ]:
m2 = Prophet(changepoints=['2017-12-16'])
m2.fit(X_train)
pred_train = m2.predict(X_train)
pred_test = m2.predict(X_test)
fig = m2.plot(pred_train)
fig = m2.plot(pred_test)

### Can you try adding more changepoints? What happens?

### 2. Adjusting trend scale

The paramater `changepoint_prior_scale` helps to control over/underfitting. Default of model is 0.5

In [ ]:
m1 = Prophet(changepoint_prior_scale=0.7)
m1.fit(X_train)
pred_train = m1.predict(X_train)
pred_test = m1.predict(X_test)
fig = m1.plot(pred_train)
fig = m1.plot(pred_test)

# 3. Configuring the seasonality

The parameter `yearly.seasonality` let's you adjust the Fourier order that models the yearly seasonality.  
The default is 10. Increasing it leads to increased model complexity, because N fourier terms corresponds to 2N variables used for modeling the cycle. Decreasing it leads to a smoother curve.

In [ ]:
m7 = Prophet(yearly_seasonality=12, weekly_seasonality=6)

**Specifying seasonalities**  

By default the Prophet will by default fit **weekly** and **yearly** seasonalities, if the time series is more than two cycles long.  
It will also fit daily seasonality for a sub-daily time series.  
You can add other seasonalities (monthly, quarterly, hourly) using the `add_seasonality` method:  

In [ ]:
# in this example we disable weekly seasonality and add a monthly instead
m6 = Prophet(weekly_seasonality=False)
m6.add_seasonality(name='monthly', period=30.5, fourier_order=5)

# 4. Add regressors 

what if we had more information about other factor that could have influenced the day-to-day trade of bitcoin?
Maybe some daily economy indicator or market conditions. Any information like this would be a timeseries itself, and it can increase the model's performance. The downside is, we need to have the data available for the future. 
If we have more series to add to the dataset, they are called **regressors**.

In [ ]:
# Add new timeseries to the dataset and then include them in the model

# Example of usage
X_train['new_regressor'] = 2 # should be a real timeseries data

m7 = Prophet()
m7.add_regressor('new_regressor')
m7.fit(X_train)

# HOMEWORK

Cover the topics **hyperparameter tuning**, **gridsearch**, **cross-validation** and **model evaluation**.
Prophet has built-in methods for all these steps. You can start checking here: https://facebook.github.io/prophet/docs/diagnostics.html 


---

<a href='https://www.freepik.com/vectors/business'>Business vector created by freepik - www.freepik.com</a>